# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import cv2
import numpy as np
import math

In [2]:
def calculate_ssd(block1, block2):
    ssd = np.sum((block1 - block2) ** 2)
    return np.sqrt(ssd)

In [3]:
def calculate_motion_vector(source_block, search_window, k):
    min_ssd = float('inf')
    best_vector = (0, 0)
    block_size = source_block.shape[0]

    for dy in range(search_window.shape[0] - block_size + 1):
        for dx in range(search_window.shape[1] - block_size + 1):
            candidate_block = search_window[dy:dy + block_size, dx:dx + block_size]
            ssd = calculate_ssd(source_block, candidate_block)
            if ssd < min_ssd:
                min_ssd = ssd
                best_vector = (dx - k, dy - k)

    return best_vector, min_ssd

In [4]:
def arrow_draw(img, x1, y1, x2, y2):
    radians = math.atan2(x1 - x2, y2 - y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10

    x11_ = x11 * math.cos(radians) - y11 * math.sin(radians) + x2
    y11_ = x11 * math.sin(radians) + y11 * math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2

    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2

    if all(isinstance(v, (int, float)) for v in [x1, y1, x2, y2, x11_, y11_, x12_, y12_, u11_, v11_, u12_, v12_]):
        img = cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 5)
        img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), (255, 0, 0), 5)
        img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), (255, 0, 0), 5)
    else:
        print("Error: Some coordinates are not valid numbers.")

    return img


In [5]:
def visualise_displacement_vectors(frame, displacement_matrix, ssd_matrix, Tmin, Tmax, k):
    for i in range(displacement_matrix.shape[0]):
        for j in range(displacement_matrix.shape[1]):
            sqrt_ssd = np.sqrt(ssd_matrix[i, j])
            if Tmin <= sqrt_ssd <= Tmax:
                x_displacement, y_displacement = displacement_matrix[i, j]
                
                x = i * (2 * k + 1) + (2 * k + 1) // 2
                y = j * (2 * k + 1) + (2 * k + 1) // 2
                
                frame = arrow_draw(frame, x, y, x + x_displacement, y + y_displacement)
                
    return frame

In [ ]:
video_path = 'monkey.avi'
cap = cv2.VideoCapture(video_path)

k = 4
block_size = 2 * k + 1
search_window_size = 15

Tmin = 10.0  
Tmax = 200.0 

ret, prev_frame = cap.read()

if ret:
    fx, fy, _ = prev_frame.shape
    displacement_vectors = np.zeros((fx // block_size, fy // block_size, 2))
    ssd_matrix = np.zeros((fx // block_size, fy // block_size))

frame_number = 1

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    fx, fy, _ = frame.shape

    for i in range(k, fx - k, block_size):
        for j in range(k, fy - k, block_size):
            block_index_x = (i - k) // block_size
            block_index_y = (j - k) // block_size

            source_block = prev_frame[i - k:i + k + 1, j - k:j + k + 1]

            search_window = frame[max(0, i - search_window_size):min(fx, i + search_window_size + 1),
                                  max(0, j - search_window_size):min(fy, j + search_window_size + 1)]

            motion_vector, ssd = calculate_motion_vector(source_block, search_window, k)
            
            if Tmin <= ssd <= Tmax:
                displacement_vector = (motion_vector[1], motion_vector[0])
            else:
                displacement_vector = (0, 0) 

            displacement_vectors[block_index_x, block_index_y] = displacement_vector
            ssd_matrix[block_index_x, block_index_y] = ssd
    
    visualised_frame = visualise_displacement_vectors(frame, displacement_vectors, ssd_matrix, Tmin, Tmax, k)

    cv2.imshow('Displacement Vectors', visualised_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    prev_frame = frame.copy()
    frame_number += 1

cap.release()
cv2.destroyAllWindows()